# Obtención de datos

Para obtener los datos vamos a usar las descripciones en cada video de las mañaneras. Pues nos provee de un resumen del video del día. De esta manera ya no tenemos que lidiar con conversaciones del presidente cuando usamos las transcripciones.

## Importaciones

In [55]:
import json 
from datetime import datetime
import re

import pandas as pd 
import numpy as np

from googleapiclient.errors import HttpError
from Google import Create_Service

## Llamamiento a la API de YouTube

<code>retrieve_playlists_items</code> Función que hace la petición a la API de youtube. Se le da un objeto <code>service</code> que es la información sobre el tipo de API y la autenticación. También recibe <code>playlistId</code> que es el ID de la playlist que queremos.

In [56]:
def retrieve_playlists_items(service, playlistId):
    items =[]
    try:
        response = service.playlistItems().list(
            part = 'contentDetails, snippet, status',
            playlistId = playlistId,
            maxResults = 5
        ).execute()

        items.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlistItems().list(
                part = 'contentDetails, snippet, status',
                playlistId = playlistId,
                maxResults = 5,
                pageToken = nextPageToken
            ).execute()

            items.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return items

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API. Toda esta es información sobre el tipo de API y los archivos necesarios para hacer la conexión con Google Cloud

In [57]:
CLIENT_SECRETS_FILE = 'client_secret.json'  # Archivo .json que se obtiene al crear las credenciales OAuth en https://console.cloud.google.com/
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [58]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 'abuseReports',
 'activities',
 'captions',
 'channelBanners',
 'channelSections',
 'channels',
 'close',
 'commentThreads',
 'comments',
 'i18nLanguages',
 'i18nRegions',
 'liveBroadcasts',
 'liveChatBans',
 'liveChatMessages',
 'liveChatModerators',
 'liveStreams',
 'members',
 'membershipsLevels',
 'new_batch_h

Hacemos una llamada a a función de <code>retrieve_playlists_items</code> para obtener todos los videos una playlist especifica. En este caso, los videos de la mañaneras

In [59]:
playlistId = 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA' # id de la playlist de las mañaneras
playlists_items = retrieve_playlists_items(service, playlistId)

In [60]:
playlists_items[0] # El último video agregado a la playlist

{'kind': 'youtube#playlistItem',
 'etag': 'Nb4URPuJT-5jBbWR-ynMdtsUf0I',
 'id': 'UExSbmxSR2FyLV8yOTZLVHNWTDBSNk1FYnB3SnpEOHBwQS4yMTMwMjBBNDJEQTM0NUQ3',
 'snippet': {'publishedAt': '2023-05-15T12:44:53Z',
  'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
  'title': 'Maestras y maestros recibirán aumento de 8.2 por ciento en su sueldo. Conferencia presidente AMLO',
  'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Lunes 15 de mayo 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n15:47 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n21:09 Comparativo de marcas y precios de combustibles\n26:28 Comparativo de marcas y precios de productos de primera necesidad\n28:45 Reporte de construcción de los Tramos 5, 6 y 7 del Tren Maya\n31:30 Informe del consorcio Mota-Engil en el Tramo 5 del Tren Maya\n33:43 Informe de la empresa Grupo Indi en el Tramo 5 del Tren Maya\n35:59 Informe de la empresa ICA en el Tramo 5 del Tren Maya\n37:36 Reporte de c

## Guardando los datos en un DataFrame

Los datos están guardados en un json. Podemos obtener cada elemento de la variable utilizando las posiciones.

In [61]:
playlists_items[0]['snippet']

{'publishedAt': '2023-05-15T12:44:53Z',
 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
 'title': 'Maestras y maestros recibirán aumento de 8.2 por ciento en su sueldo. Conferencia presidente AMLO',
 'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Lunes 15 de mayo 2023 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n15:47 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n21:09 Comparativo de marcas y precios de combustibles\n26:28 Comparativo de marcas y precios de productos de primera necesidad\n28:45 Reporte de construcción de los Tramos 5, 6 y 7 del Tren Maya\n31:30 Informe del consorcio Mota-Engil en el Tramo 5 del Tren Maya\n33:43 Informe de la empresa Grupo Indi en el Tramo 5 del Tren Maya\n35:59 Informe de la empresa ICA en el Tramo 5 del Tren Maya\n37:36 Reporte de construcción de la Sedena en los Tramos 5, 6 y 7 del Tren Maya\n37:36 Obras complementarias a cargo de la Sedena en los Tramos 5, 6 y 7 del Tren Maya\n42:40 Informe de 

Extraeremos sólo la información necesaria para hacer el entrenamiento. Estos datos son: título, descripción y la fecha de publicación

In [62]:

data = []

for item in playlists_items:
    snippet = item['snippet']
    published_at = snippet['publishedAt']
    title = snippet['title']
    description = snippet['description']
    
    data.append([published_at, title, description])
    
df = pd.DataFrame(data, columns=['published_at', 'title', 'description'])
df = df.head(1099)

print(df)


              published_at                                              title   
0     2023-05-15T12:44:53Z  Maestras y maestros recibirán aumento de 8.2 p...  \
1     2023-05-12T03:54:24Z  Gobierno no permitirá insultos de republicanos...   
2     2023-05-11T12:44:34Z  Sueldos de magistrados y jueces deben bajar pa...   
3     2023-05-10T12:50:23Z  Homenaje a las madres de México en su día. Con...   
4     2023-05-09T12:47:41Z  Fin de la emergencia sanitaria por COVID-19 en...   
...                    ...                                                ...   
1094  2019-03-08T21:59:25Z          Iniciamos la semana dialogando con medios   
1095  2019-03-08T21:59:25Z  Conferencia matutina: Avances del Gabinete de ...   
1096  2019-03-08T21:59:25Z  Hoy anunciamos la terna de candidatos a minist...   
1097  2019-03-08T21:59:25Z  Tercera conferencia de prensa matutina desde P...   
1098  2019-03-08T21:59:25Z     Diálogo con periodistas desde Palacio Nacional   

                           

## Una primera limpieza

Los datos que llegan de la API están contaminados con información como minutos de YouTube en formato <code>00:00</code>, links, saltos de línea <code>\n</code> y palabras específicas como nombres redes sociales o frases que no aportan nada.
Para mejor ejemplo, tomemos un elemento.

In [63]:
df['description'][500]

'Conferencia de prensa matutina, desde Palacio Nacional. Jueves 15 de abril 2021 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n14:49 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n16:00 Sesión de preguntas y respuestas\n\nCelebramos la llegada de dos embarques de vacunas contra #COVID19 a la Ciudad de México; 500 mil dosis de la vacuna SinoVac y 487 mil 500 de Pfizer BioNTech.\n\nSe utilizarán para aplicar segundas dosis y vacunar a personas de 50 a 59 años. Mañana informaremos sobre la vacunación al personal educativo de escuelas públicas y privadas para regresar a clases presenciales antes de terminar el ciclo escolar, teniendo claro que el retorno a las aulas será voluntario. \n\nCon las 987 mil 500 que hoy recibimos, México suma un total de 17 millones 888 mil 350 dosis de las vacunas Cansino, Sputnik V, Sinovac, AstraZeneca y Pfizer. \n\nLas 32 entidades federativas ya cuentan con las dosis necesarias para terminar la vacunación a adultos mayores

Declararemos una función <code>replace_patterns</code>que recibirá una row y una serie de patrones que contendrán el regex a buscar y con lo que se va a reeemplazar

In [64]:
def replace_patterns(row, patterns):
    text = row['description']
    for pattern, replacement in patterns.items():
        regex = re.compile(pattern, re.IGNORECASE | re.DOTALL)
        text = regex.sub(replacement, text)
    return text


La función <code>remove_beginning</code> se creó principamente porque el regex estaba dando problemas si no se colocaba en una función, por su complejidad probablemente.

In [65]:
def remove_beginning(string):
    pattern = r'Conferencia de prensa (matutina|en vivo),?( desde (\b\w+\b,?\s)?(\b\w+\b\s)?\b\w+\b)\. \b(Lunes|Martes|Miércoles|Jueves|Viernes|Sábado|Domingo)\b [0-3]?[0-9] de \b(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)\b 20[0-9][0-9]\.?'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

Aquí está los patrones que se buscarán y con lo que se va a reemplazar

In [66]:
patterns = {

    # Borra palabras clave dentro de la descripción
    "Sigue las actividades del presidente de México:": "",
    "\s+Más información:": "",
    "Sitio web:\s+": "",
    "YouTube:\s+": "",
    "Facebook:\s+": "",
    "Twitter:\s+": "",
    "Instagram:\s+": "",
    "Telegram:\s+": "",
    "Spotify:": "",
    "#EnVivo\s+": "",
    "#AMLO": "",
    
    # Borrar separadores
    "\|": "", 

    # Borra saltos de línea:
    "\n": " ", 

    # Borra los saltos a minutos en YouTube:
    "[0-9]?[0-9]?:[0-9][0-9]": "", 

    # Borra cualquier link de internet:
    "http(s)?://[a-zA-Z0-9./?=_-]+": "",

    # Elemina todos los # (hashtags)
    "#\w+": "",

    #Elimina todas las palabras con @ al principio
    "@\w+": "",

    # Eliminando links a redes sociales
    "\w+.com/\w+.\w+.\w+": "",

    #Eliminando la palabra lopezobrador
    "lopezobrador": "",

    # Borra el inicio la mención del presidente
    "Presidente AMLO.": "",

    # Borra la el inicio de la transmisión
    "Inicio de transmisión": "",
    "Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador": "",

    # Los espacios dobles se reemplazan por un solo espacio
    "  +": " "

}   

Aplicamos las funciones y vemos los resultados

In [67]:
df['description'] = df.apply(replace_patterns, patterns=patterns, axis=1).apply(remove_beginning)

In [68]:
df['description'][500]

' Sesión de preguntas y respuestas Celebramos la llegada de dos embarques de vacunas contra a la Ciudad de México; 500 mil dosis de la vacuna SinoVac y 487 mil 500 de Pfizer BioNTech. Se utilizarán para aplicar segundas dosis y vacunar a personas de 50 a 59 años. Mañana informaremos sobre la vacunación al personal educativo de escuelas públicas y privadas para regresar a clases presenciales antes de terminar el ciclo escolar, teniendo claro que el retorno a las aulas será voluntario. Con las 987 mil 500 que hoy recibimos, México suma un total de 17 millones 888 mil 350 dosis de las vacunas Cansino, Sputnik V, Sinovac, AstraZeneca y Pfizer. Las 32 entidades federativas ya cuentan con las dosis necesarias para terminar la vacunación a adultos mayores durante el mes de abril. Estamos satisfechos con la actuación de la Guardia Nacional, congruente con los principios de rechazo a la violencia de género. Los valores de esta institución y de las Fuerzas Armadas permitirán acercarlas más al pu

## Separación por semana

Ahora necesitamos separar por semana, es decir, juntar todas las descripciones de una semana. De esta manera cada renglón del dataframe reprensentará una semana en específico.
Lo primero que debemos hacer es limpiar el apartado de la fecha y convertirlo a datetime.

In [69]:
df['published_at']

0       2023-05-15T12:44:53Z
1       2023-05-12T03:54:24Z
2       2023-05-11T12:44:34Z
3       2023-05-10T12:50:23Z
4       2023-05-09T12:47:41Z
                ...         
1094    2019-03-08T21:59:25Z
1095    2019-03-08T21:59:25Z
1096    2019-03-08T21:59:25Z
1097    2019-03-08T21:59:25Z
1098    2019-03-08T21:59:25Z
Name: published_at, Length: 1099, dtype: object

Lo que se haces es: tomar los primeros 10 digitos de cada elemento de la columna y convertirlo al tipo datetime con la función <code>datetime.strptime</code>. Son los primeros 10 digitos porque son los que representan la fecha de ese día, no nos importan las horas, minutos o segundos así que lo descartamos

In [70]:
clean_dates = []

for dates in df['published_at']:
    clean_dates.append( datetime.strptime(dates[:10], '%Y-%m-%d') )
df['clean_dates'] = clean_dates

In [71]:
df = df.drop('published_at', axis=1) # Ahorra borramos las fechas que no nos sirven

In [72]:
df['clean_dates'] = pd.to_datetime(df['clean_dates'])

Ya tenemos los datos como fechas tipo <code>datetime</code>

In [73]:
df

,title,description,clean_dates
0,Maestras y maestros recibirán aumento de 8.2 p...,Comparativo de marcas y precios de combustibl...,2023-05-15
1,Gobierno no permitirá insultos de republicanos...,Informe de la Segob sobre permisos para la op...,2023-05-12
2,Sueldos de magistrados y jueces deben bajar pa...,Fomento al deporte a través del Programa de M...,2023-05-11
3,Homenaje a las madres de México en su día. Con...,Reporte sobre noticias falsas Sesión de pregu...,2023-05-10
4,Fin de la emergencia sanitaria por COVID-19 en...,Anuncio del fin de la emergencia sanitaria po...,2023-05-09
...,...,...,...
1094,Iniciamos la semana dialogando con medios,Esta mañana dimos a conocer el trabajo coordin...,2019-03-08
1095,Conferencia matutina: Avances del Gabinete de ...,Hoy destacamos la conformación de un sistema ...,2019-03-08
1096,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...,2019-03-08
1097,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...,2019-03-08


In [74]:
df['clean_dates']

0      2023-05-15
1      2023-05-12
2      2023-05-11
3      2023-05-10
4      2023-05-09
          ...    
1094   2019-03-08
1095   2019-03-08
1096   2019-03-08
1097   2019-03-08
1098   2019-03-08
Name: clean_dates, Length: 1099, dtype: datetime64[ns]

La mejor manera de manipular datos con fechas es creando una serie de tiempo, para estos se utiliza la libería <code>datetime</code>. Convertimos el Data Frame en una serie de tiempo con <code>pd.set_index</code>.
De esta manera las fechas ahora serán los índices de el dataframe

In [75]:
df.set_index('clean_dates', inplace=True)

In [76]:
df

,title,description
clean_dates,,
2023-05-15,Maestras y maestros recibirán aumento de 8.2 p...,Comparativo de marcas y precios de combustibl...
2023-05-12,Gobierno no permitirá insultos de republicanos...,Informe de la Segob sobre permisos para la op...
2023-05-11,Sueldos de magistrados y jueces deben bajar pa...,Fomento al deporte a través del Programa de M...
2023-05-10,Homenaje a las madres de México en su día. Con...,Reporte sobre noticias falsas Sesión de pregu...
2023-05-09,Fin de la emergencia sanitaria por COVID-19 en...,Anuncio del fin de la emergencia sanitaria po...
...,...,...
2019-03-08,Iniciamos la semana dialogando con medios,Esta mañana dimos a conocer el trabajo coordin...
2019-03-08,Conferencia matutina: Avances del Gabinete de ...,Hoy destacamos la conformación de un sistema ...
2019-03-08,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...


In [77]:
df['description'].to_csv('dataframe.csv')

In [78]:
df.resample('W').sum()

,title,description
clean_dates,,
2019-03-10,Presidente AMLO llama a terminar con feminicid...,Hoy conmemoramos la lucha de las mujeres por ...
2019-03-17,Reformas constitucionales aprobadas representa...,Hoy dimos a conocer el análisis de la recient...
2019-03-24,Presidente firma compromiso de no reelección; ...,Hoy dimos a conocer que ya firmamos la carta ...
2019-03-31,Acciones para mejorar protección a defensores ...,Hoy informamos sobre las acciones para mejora...
2019-04-07,Avanza entrega de apoyos de 'Programas Integra...,En el Gobierno de México avanzamos con entusi...
...,...,...
2023-04-23,Avance mayor al 60 por ciento en obras del tra...,Comparativo de marcas y precios de combustibl...
2023-04-30,Conferencia de prensa matutina desde Palacio N...,Comparativo de marcas y precios de combustibl...
2023-05-07,Tarjeta Finabien para mexicanos en Estados Uni...,Comparativo de marcas y precios de combustibl...


Ahora le decimos que separe, con respecto a las fechas, semanalmente. De esta manera va a concatenar los contenidos de las descripciones y los títulos.

In [79]:
weekly_ts = df.resample('W').sum()

Se muestra el dataframe

In [80]:
weekly_ts

,title,description
clean_dates,,
2019-03-10,Presidente AMLO llama a terminar con feminicid...,Hoy conmemoramos la lucha de las mujeres por ...
2019-03-17,Reformas constitucionales aprobadas representa...,Hoy dimos a conocer el análisis de la recient...
2019-03-24,Presidente firma compromiso de no reelección; ...,Hoy dimos a conocer que ya firmamos la carta ...
2019-03-31,Acciones para mejorar protección a defensores ...,Hoy informamos sobre las acciones para mejora...
2019-04-07,Avanza entrega de apoyos de 'Programas Integra...,En el Gobierno de México avanzamos con entusi...
...,...,...
2023-04-23,Avance mayor al 60 por ciento en obras del tra...,Comparativo de marcas y precios de combustibl...
2023-04-30,Conferencia de prensa matutina desde Palacio N...,Comparativo de marcas y precios de combustibl...
2023-05-07,Tarjeta Finabien para mexicanos en Estados Uni...,Comparativo de marcas y precios de combustibl...


Ahora se guarda el dataframe en un .txt para que el preprocesamiento pueda llamarlo y utlizarlo

In [81]:
weekly_ts['description'].to_csv('mañaneras.csv')

Así es como se termina la obtención de datos, ya tenemos nuesto dataframe listo para mandar al preprocesamiento. Como se observó, a los datos se les hizo una primera limpieza, la razón de esto fue para que en el preprocesamiento se hicieran los pasos fundamentales y no pasos extras, como fue este caso de quitar ciertos patrones que estorbaban.